In [ ]:
class FileBase:
    def __init__(self, filename=None, mode='r'):
        self.file = None
        self.filename = filename
        self.mode = mode

    def open(self, filename, mode='r'):
        self.close()
        self.filename = filename
        self.mode = mode
        try:
            self.file = open(self.filename, self.mode)
            return True
        except IOError:
            print("Unable to open file.")
            return False

    def close(self):
        if self.file is not None:
            self.file.close()
            self.file = None

    def is_eof(self):
        return self.file.tell() == os.fstat(self.file.fileno()).st_size

class FileLoader(FileBase):
    def __init__(self, filename=None, columns=None, filetype='TEXT'):
        super().__init__(filename, 'r' if filetype == 'TEXT' else 'rb')
        self.columns = columns
        self.filetype = filetype
        self.data = []

    def load(self):
        if not self.load_():
            return []
        return self.data

    def loadn(self, epochs):
        datas = []
        for _ in range(epochs):
            if not self.load_():
                break
            datas.append(self.data.copy())
        return datas

    def load_(self):
        if self.is_eof():
            return False

        if self.filetype == 'TEXT':
            line = self.file.readline()
            if not line:
                return False

            # Handling whitespace and splitting by comma or whitespace
            splits = [x.strip() for x in line.split(',') if x.strip()]
            self.data = [float(val) for val in splits]
        else:
            import struct
            data_bytes = self.file.read(self.columns * 8)  # assuming double precision floats
            if not data_bytes:
                return False
            self.data = list(struct.unpack('d' * self.columns, data_bytes))
        return True

# Constants for file types
TEXT = 'TEXT'
BINARY = 'BINARY'

# Example usage
loader = FileLoader('data.txt', 5, TEXT)
print(loader.loadn(10))
loader.close()
